## MAPAS json MADRID

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
eventos = pd.read_csv('../csv/eventos_def.csv')
restaurantes = pd.read_csv('../csv/rest_clean.csv')
geo = gpd.read_file('./barrios-madrid.geojson')

- LOCALIZO EL PUNTO XY DONDE SE ENCUENTRA LA LOCALIZACION EN EL MAPA. 
- POSTERIORMENTE, BUSCO EL BARRIO SEGÚN LAS COORDENADAS. 

### Geolocalizacion de Restaurantes

In [3]:
restaurantes.head()

,restaurante,gastronomia,precio,opinion,reseña,ubicacion,lat,long
0,Yakiniku Rikyu,Japonesa,Medio,267 opiniones,Patrocinado,"P.º de la Castellana, 15, 28046 Madrid",40.4280979,-3.690173
1,marmitón,Mediterránea,Medio,89 opiniones,MICHELIN,"C. de las Aguas, 6, 28005 Madrid",40.4108219,-3.712107746868964
2,Vinoteca Moratín,Internacional,Medio,1221 opiniones,MICHELIN,"C. de Moratín, 36, 28014 Madrid",40.412472050000005,-3.6954412113077497
3,Pilar Akaneya,Japonesa,Alto,276 opiniones,MICHELIN,"C. de Espronceda, 33, 28003 Madrid",40.4405573,-3.696109
4,Gioia,Italiana,Medio,387 opiniones,MICHELIN,"C. de San Bartolomé, 23, 28004 Madrid",40.4221651,-3.6983792


In [4]:
restaurantes.insert(6, 'punto', 0)

In [5]:
len(restaurantes)

300

In [6]:
# Identificar y manejar los valores no válidos
mask = (restaurantes['long'] == 'No disponible') | (restaurantes['lat'] == 'No disponible')

In [7]:
# Opción 1: Eliminar filas con valores no válidos en cuanto a que no tienen long / lat disponible.
rest = restaurantes[~mask]

In [8]:
# veo como el número de restaurantes ha disminuido, ya que tenemos muchos datos en los que no hemos encontrado su ubicación o no nos ha funcionado el geopy. 
# Mejorar la limpieza de las localizaciones con la herramienta Geopy será un NEXT STEP en mi trabajo. 

len(rest)

193

In [9]:
len(restaurantes)

300

In [10]:
rest.head()

,restaurante,gastronomia,precio,opinion,reseña,ubicacion,punto,lat,long
0,Yakiniku Rikyu,Japonesa,Medio,267 opiniones,Patrocinado,"P.º de la Castellana, 15, 28046 Madrid",0,40.4280979,-3.690173
1,marmitón,Mediterránea,Medio,89 opiniones,MICHELIN,"C. de las Aguas, 6, 28005 Madrid",0,40.4108219,-3.712107746868964
2,Vinoteca Moratín,Internacional,Medio,1221 opiniones,MICHELIN,"C. de Moratín, 36, 28014 Madrid",0,40.412472050000005,-3.6954412113077497
3,Pilar Akaneya,Japonesa,Alto,276 opiniones,MICHELIN,"C. de Espronceda, 33, 28003 Madrid",0,40.4405573,-3.696109
4,Gioia,Italiana,Medio,387 opiniones,MICHELIN,"C. de San Bartolomé, 23, 28004 Madrid",0,40.4221651,-3.6983792


- Como de esta forma perderíamos muchos valores de restaurantes, lo que voy a hacer es reemplazar los valores no válidos con NaN. 

In [11]:
# Opción 2: Reemplazar los valores no válidos con NaN
restaurantes['long'] = pd.to_numeric(restaurantes['long'], errors='coerce')
restaurantes['lat'] = pd.to_numeric(restaurantes['lat'], errors='coerce')

In [12]:
# Crear la GeoSeries
restaurantes['punto'] = gpd.GeoSeries.from_xy(restaurantes['long'], restaurantes['lat'])

In [13]:
restaurantes['punto'].head()

0    POINT (-3.69017 40.42810)
1    POINT (-3.71211 40.41082)
2    POINT (-3.69544 40.41247)
3    POINT (-3.69611 40.44056)
4    POINT (-3.69838 40.42217)
Name: punto, dtype: geometry

In [14]:
len(restaurantes)

300

In [15]:
barrio_rest = []

for p in restaurantes.punto:

    encontrado = False

    for i, area in enumerate(geo.geometry):
        if area.contains(p):
            barrio_rest.append(geo.name[i])
            encontrado = True
            break
            
    if not encontrado:
        barrio_rest.append("No encontrado")

In [16]:

restaurantes.insert(7, 'barrio', barrio_rest)

In [17]:
restaurantes.head()

,restaurante,gastronomia,precio,opinion,reseña,ubicacion,punto,barrio,lat,long
0,Yakiniku Rikyu,Japonesa,Medio,267 opiniones,Patrocinado,"P.º de la Castellana, 15, 28046 Madrid",POINT (-3.69017 40.42810),Almagro,40.428098,-3.690173
1,marmitón,Mediterránea,Medio,89 opiniones,MICHELIN,"C. de las Aguas, 6, 28005 Madrid",POINT (-3.71211 40.41082),Palacio,40.410822,-3.712108
2,Vinoteca Moratín,Internacional,Medio,1221 opiniones,MICHELIN,"C. de Moratín, 36, 28014 Madrid",POINT (-3.69544 40.41247),Cortes,40.412472,-3.695441
3,Pilar Akaneya,Japonesa,Alto,276 opiniones,MICHELIN,"C. de Espronceda, 33, 28003 Madrid",POINT (-3.69611 40.44056),Rios Rosas,40.440557,-3.696109
4,Gioia,Italiana,Medio,387 opiniones,MICHELIN,"C. de San Bartolomé, 23, 28004 Madrid",POINT (-3.69838 40.42217),Justicia,40.422165,-3.698379


## Geolocalizacion de Eventos

In [18]:
eventos.head()

,nombre,local,precio,enlace,año,mes,dia,hora,ubicacion,lat,long
0,El Rey León,Teatro Lope de Vega,49.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,11,30,19:00,"C.Gran Vía, 57, 28013 Madrid",40.4217903,-3.7086754
1,El Rey León,Teatro Lope de Vega,45.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,1,17:00,"C.Gran Vía, 57, 28013 Madrid",40.4217903,-3.7086754
2,El Rey León,Teatro Lope de Vega,45.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,1,21:00,"C.Gran Vía, 57, 28013 Madrid",40.4217903,-3.7086754
3,El Rey León,Teatro Lope de Vega,67.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,2,17:00,"C.Gran Vía, 57, 28013 Madrid",40.4217903,-3.7086754
4,El Rey León,Teatro Lope de Vega,45.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,2,21:00,"C.Gran Vía, 57, 28013 Madrid",40.4217903,-3.7086754


In [19]:
eventos.insert(9, 'punto', 0)

In [20]:
eventos.head()

,nombre,local,precio,enlace,año,mes,dia,hora,ubicacion,punto,lat,long
0,El Rey León,Teatro Lope de Vega,49.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,11,30,19:00,"C.Gran Vía, 57, 28013 Madrid",0,40.4217903,-3.7086754
1,El Rey León,Teatro Lope de Vega,45.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,1,17:00,"C.Gran Vía, 57, 28013 Madrid",0,40.4217903,-3.7086754
2,El Rey León,Teatro Lope de Vega,45.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,1,21:00,"C.Gran Vía, 57, 28013 Madrid",0,40.4217903,-3.7086754
3,El Rey León,Teatro Lope de Vega,67.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,2,17:00,"C.Gran Vía, 57, 28013 Madrid",0,40.4217903,-3.7086754
4,El Rey León,Teatro Lope de Vega,45.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,2,21:00,"C.Gran Vía, 57, 28013 Madrid",0,40.4217903,-3.7086754


In [21]:
# Opción 2: Reemplazar los valores no válidos con NaN
eventos['long'] = pd.to_numeric(eventos['long'], errors='coerce')
eventos['lat'] = pd.to_numeric(eventos['lat'], errors='coerce')

In [22]:
# Crear la GeoSeries
eventos['punto'] = gpd.GeoSeries.from_xy(eventos['long'], eventos['lat'])

In [23]:
eventos['punto'].head()

0    POINT (-3.70868 40.42179)
1    POINT (-3.70868 40.42179)
2    POINT (-3.70868 40.42179)
3    POINT (-3.70868 40.42179)
4    POINT (-3.70868 40.42179)
Name: punto, dtype: geometry

In [24]:
barrio_event = []

for p in eventos.punto:

    encontrado = False

    for i, area in enumerate(geo.geometry):
        if area.contains(p):
            barrio_event.append(geo.name[i])
            encontrado = True
            break
            
    if not encontrado:
        barrio_event.append("No encontrado")

In [25]:
eventos.insert(10, 'barrio', barrio_event)

In [26]:
eventos.head()

,nombre,local,precio,enlace,año,mes,dia,hora,ubicacion,punto,barrio,lat,long
0,El Rey León,Teatro Lope de Vega,49.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,11,30,19:00,"C.Gran Vía, 57, 28013 Madrid",POINT (-3.70868 40.42179),Palacio,40.42179,-3.708675
1,El Rey León,Teatro Lope de Vega,45.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,1,17:00,"C.Gran Vía, 57, 28013 Madrid",POINT (-3.70868 40.42179),Palacio,40.42179,-3.708675
2,El Rey León,Teatro Lope de Vega,45.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,1,21:00,"C.Gran Vía, 57, 28013 Madrid",POINT (-3.70868 40.42179),Palacio,40.42179,-3.708675
3,El Rey León,Teatro Lope de Vega,67.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,2,17:00,"C.Gran Vía, 57, 28013 Madrid",POINT (-3.70868 40.42179),Palacio,40.42179,-3.708675
4,El Rey León,Teatro Lope de Vega,45.0,https://www.entradas.com/event/el-rey-leon-el-...,2023,12,2,21:00,"C.Gran Vía, 57, 28013 Madrid",POINT (-3.70868 40.42179),Palacio,40.42179,-3.708675


In [27]:
#Guardo el df completo de cada uno a un nuevo .csv

restaurantes.to_csv('../csv/restaurante_fin.csv')
eventos.to_csv('../csv/eventos_fin.csv')